In [21]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
import numpy as np
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd

In [22]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
import numpy as nAp
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [23]:
from shutil import copyfile
from os import getcwd

In [24]:
ZIP_DIR = "D:/Campus/4 Sem Files/Programming Challenge II - CS2212/"
DATASET_DIR = "D:/Campus/4 Sem Files/Programming Challenge II - CS2212/kvasir-dataset-v2/"
SOURCE_DIR = "D:/Campus/4 Sem Files/programming Challenge II - CS2212/dataset/"
WEIGHTS_DIR = "D:/Campus/4 Sem Files/programming Challenge II - CS2212/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"


In [25]:
#change the current working directory
os.chdir(ZIP_DIR)
path_dataset = getcwd()
print(path_dataset)
path_dataset = f"{getcwd()}/kvasir-dataset-v2.zip"
print(path_dataset)

# #Delete the folders if they already exist
if os.path.exists(DATASET_DIR) and os.path.isdir(DATASET_DIR):
    shutil.rmtree(DATASET_DIR)

if os.path.exists(SOURCE_DIR) and os.path.isdir(SOURCE_DIR):
    shutil.rmtree(SOURCE_DIR)

local_zip = path_dataset
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall(ZIP_DIR)
zip_ref.close()

D:\Campus\4 Sem Files\Programming Challenge II - CS2212
D:\Campus\4 Sem Files\Programming Challenge II - CS2212/kvasir-dataset-v2.zip


In [26]:
if not (os.path.exists(SOURCE_DIR)) or not (os.path.isdir(SOURCE_DIR)):
    os.mkdir(SOURCE_DIR)
    os.mkdir(SOURCE_DIR + "/training")
    os.mkdir(SOURCE_DIR + "/testing")


In [27]:
labels = os.listdir(DATASET_DIR)
print(labels)

for label in labels:
    test_path = SOURCE_DIR+ "/training/"+label
    train_path = SOURCE_DIR + "/testing/"+label

    if not (os.path.exists(train_path)) or not (os.path.isdir(train_path)):
        os.mkdir(train_path)

    if not (os.path.exists(test_path)) or not (os.path.isdir(test_path)):
        os.mkdir(test_path)

        

['dyed-lifted-polyps', 'dyed-resection-margins', 'esophagitis', 'normal-cecum', 'normal-pylorus', 'normal-z-line', 'polyps', 'ulcerative-colitis']


In [29]:
for label in labels:
    print(label,end=" : ")
    print(len(os.listdir(DATASET_DIR+label)))


dyed-lifted-polyps : 1000
dyed-resection-margins : 1000
esophagitis : 1000
normal-cecum : 1000
normal-pylorus : 1000
normal-z-line : 1000
polyps : 1000
ulcerative-colitis : 1000


In [30]:
#split the dataset into trianing and testing set according to split_size
def split_data(SOURCE, TRAINING, TESTING, SPLIT_RATIO):
    labels = os.listdir(SOURCE)
    
    for label in labels:
        
        source_path = SOURCE + label +'/'
        training_path = TRAINING + label +'/'
        testing_path = TESTING + label +'/'
        
        data = os.listdir(source_path)
        filtered_data = []
        
        for image in data:
            if(os.path.getsize(source_path+image)):
                filtered_data.append(image)

        
        random.sample(filtered_data,len(filtered_data))
        train_set = filtered_data[:int(len(filtered_data)*SPLIT_RATIO)]
        test_set  = filtered_data[int(len(filtered_data)*SPLIT_RATIO):]

        for train_image in train_set:
            copyfile(source_path + train_image,training_path + train_image)

        for test_image in test_set:
            copyfile(source_path + test_image,testing_path + test_image)


TRAINING_DIR = SOURCE_DIR + "/training/"
TESTING_DIR = SOURCE_DIR  + "/testing/"

split_ratio = .75
split_data(DATASET_DIR, TRAINING_DIR, TESTING_DIR, split_ratio)


In [31]:
for label in labels:
    print(label)
    print("train", end=" : ")
    print(len(os.listdir(SOURCE_DIR+ "/training/"+label)))
    print("test", end=" : ")
    print(str(len(os.listdir(SOURCE_DIR+ "/testing/"+label)))+"\n")


dyed-lifted-polyps
train : 750
test : 250

dyed-resection-margins
train : 750
test : 250

esophagitis
train : 750
test : 250

normal-cecum
train : 750
test : 250

normal-pylorus
train : 750
test : 250

normal-z-line
train : 750
test : 250

polyps
train : 750
test : 250

ulcerative-colitis
train : 750
test : 250



In [32]:
pre_trained_model = InceptionV3(input_shape = (224, 224, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(WEIGHTS_DIR)



for layer in pre_trained_model.layers[:-10]:
  layer.trainable = False


last_output = pre_trained_model.layers[-1].output



In [33]:
x = layers.Flatten()(last_output)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)                  
x = layers.Dense  (8, activation='softmax')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])

In [34]:
TRAINING_DIR = SOURCE_DIR + "/training/"
train_datagen = ImageDataGenerator(rescale =1.0/255,fill_mode='nearest', width_shift_range=0.2, height_shift_range=0.2,brightness_range= [0.4,1],shear_range=0.2,zoom_range= [0.8,1.4])

    

train_generator = train_datagen.flow_from_directory(TRAINING_DIR,batch_size = 32, class_mode='categorical',target_size = (224,224), )

VALIDATION_DIR = SOURCE_DIR + "/testing/"
validation_datagen = ImageDataGenerator(rescale =1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,batch_size = 32, class_mode='categorical',target_size = (224,224))



Found 6000 images belonging to 8 classes.
Found 2000 images belonging to 8 classes.


In [35]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.97):
      print("\nReached 97.0% accuracy so cancelling training!")
      self.model.stop_training = True


In [ ]:
callbacks = myCallback()
history = model.fit(train_generator,
                              epochs=50,
                              verbose=1,
                              validation_data=validation_generator,
                              callbacks = [callbacks]
                    )


Epoch 1/50
188/188 [==============================] - 714s 4s/step - loss: 1.5103 - acc: 0.4580 - val_loss: 0.7533 - val_acc: 0.7255
Epoch 2/50
188/188 [==============================] - 753s 4s/step - loss: 1.0467 - acc: 0.6078 - val_loss: 0.6129 - val_acc: 0.7615
Epoch 3/50
188/188 [==============================] - 799s 4s/step - loss: 0.9286 - acc: 0.6527 - val_loss: 0.5372 - val_acc: 0.7950
Epoch 4/50
188/188 [==============================] - 691s 4s/step - loss: 0.8985 - acc: 0.6665 - val_loss: 0.4663 - val_acc: 0.8185
Epoch 5/50
188/188 [==============================] - 728s 4s/step - loss: 0.8265 - acc: 0.6887 - val_loss: 0.4729 - val_acc: 0.8185
Epoch 6/50
113/188 [=================>............] - ETA: 3:53 - loss: 0.7953 - acc: 0.7008

In [ ]:
# PLOT LOSS AND ACCURACY
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")


plt.title('Training and validation loss')

# Desired output. Charts with training and validation metrics. No crash :)